# $\text{Get PDF on Server}$

# $\text{PDF Cleaning}$

We extract cigrarettes product characteristics from [Federal Trade Commission Report of Cigarettes (1998)](https://www.ftc.gov/sites/default/files/documents/reports/2000-report-tar-nicotine-and-carbon-monoxide-covering-1998/1998tarnicotinereport_0.pdf).

In [110]:
# save as extract_ftc_tnco_v2.py
# run: python extract_ftc_tnco_v2.py /path/to/1998tarnicotinereport_0.pdf
import sys, re, csv, os
import pdfplumber

PDF = "Characteristics/1998tarnicotinereport_0.pdf"
OUT = "Characteristics/ftc_1998_tnco_clean.csv"
PAGE_START, PAGE_END = 10, 45  # adjust if needed

HDR_RE    = re.compile(r'(?i)\bbrand\s+name\b.*\bdescription\b.*\btar\b.*\bnic\b.*\bco\b')
NUMTOK_RE = re.compile(r'^(?:<)?\d+(?:\.\d+)?$|^NA$', re.I)

# Known tokens that indicate we've reached characteristics (not brand words)
SIZE_TOKS   = {"100","100s","100's","KING","KINGS","REG","REGULAR"}
PACK_TOKS   = {"HP","SP"}  # hard/soft pack abbreviations used in the report
FILTER_TOKS = {"F","NF"}
STRENGTH_TOKS = {"FF","FULL","FLAVOR","LIGHT","LT","ULTRA","ULTRA-LT","ULTIMA","MILD"}
OTHER_CHAR_TOKS = {"MENTHOL","DLX","DELUXE","SPECIAL","SUP","SLIM","SUPERSLIM","SUPER","BOX","BOXES","GENERIC"}

CHAR_STARTERS = SIZE_TOKS | PACK_TOKS | FILTER_TOKS | STRENGTH_TOKS | OTHER_CHAR_TOKS

# Value/generic-style brands to flag (you can extend this)
VALUE_BRANDS = {
    "BEST BUY", "BARGAIN BUY", "ALL AMERICAN VALUE", "ALL-AMERICAN VALUE",
    "AMERICAN VALUE"
}

def find_header_cuts(page):
    words = page.extract_words(use_text_flow=True, keep_blank_chars=False)
    header_words = [w for w in words if w["text"].lower() in {"brand","name","description","tar","nic","co"}]
    if not header_words:
        return None, None, None

    # group header words by line
    y_key = round(sorted(header_words, key=lambda w: w["top"])[0]["top"]/3.0)
    ws = [w for w in header_words if round(w["top"]/3.0)==y_key]

    # x starts
    def x(label):
        xs = [w["x0"] for w in ws if w["text"].lower()==label]
        return min(xs) if xs else None

    x_brand = x("brand"); x_name = x("name"); x_desc = x("description")
    x_tar   = x("tar");   x_nic  = x("nic");  x_co   = x("co")
    if None in (x_brand,x_name,x_desc,x_tar,x_nic,x_co):
        return None, None, None

    x_brand_start = min(x_brand, x_name)
    x_brand_end   = (x_desc + x_brand_start)/2.0
    x_desc_end    = (x_tar + x_desc)/2.0
    x_tar_end     = (x_nic + x_tar)/2.0
    x_nic_end     = (x_co + x_nic)/2.0

    header_top    = min(w["top"] for w in ws)
    header_bottom = max(w["bottom"] for w in ws)
    return (x_brand_start, x_brand_end, x_desc_end, x_tar_end, x_nic_end), header_top, header_bottom

def bucketize(words, cuts, header_bottom):
    x_brand_start, x_brand_end, x_desc_end, x_tar_end, x_nic_end = cuts
    lines = {}
    for w in words:
        y_mid = (w["top"] + w["bottom"]) / 2.0
        if y_mid <= header_bottom + 1.0:
            continue
        key = round(y_mid / 2.0, 1)
        lines.setdefault(key, []).append(w)

    rows = []
    for key in sorted(lines.keys()):
        ws = sorted(lines[key], key=lambda z: z["x0"])
        cols = {"brand": [], "desc": [], "tar": [], "nic": [], "co": []}
        for w in ws:
            xm = (w["x0"] + w["x1"]) / 2.0
            txt = w["text"]
            if xm < x_brand_end:
                cols["brand"].append(txt)
            elif xm < x_desc_end:
                cols["desc"].append(txt)
            elif xm < x_tar_end:
                cols["tar"].append(txt)
            elif xm < x_nic_end:
                cols["nic"].append(txt)
            else:
                cols["co"].append(txt)

        brand = " ".join(cols["brand"]).strip()
        desc  = " ".join(cols["desc"]).strip()
        tar   = " ".join(cols["tar"]).strip()
        nic   = " ".join(cols["nic"]).strip()
        co    = " ".join(cols["co"]).strip()

        # If TNCO aren't clean, try right-anchored salvage
        if not (NUMTOK_RE.match(tar or "") and NUMTOK_RE.match(nic or "") and NUMTOK_RE.match(co or "")):
            toks = (brand + "  " + desc + "  " + tar + " " + nic + " " + co).split()
            idxs = [i for i,t in enumerate(toks) if NUMTOK_RE.match(t)]
            if len(idxs) >= 3:
                i3,i2,i1 = idxs[-1],idxs[-2],idxs[-3]
                co,nic,tar = toks[i3], toks[i2], toks[i1]
                left = " ".join(toks[:i1])
                m = re.search(r'\s{2,}', left)
                if m:
                    brand = left[:m.start()].strip()
                    desc  = left[m.end():].strip()
                else:
                    parts = left.split()
                    brand = parts[0] if parts else ""
                    desc  = " ".join(parts[1:]) if len(parts)>1 else ""

        if NUMTOK_RE.match(tar or "") and NUMTOK_RE.match(nic or "") and NUMTOK_RE.match(co or ""):
            rows.append([brand, desc, tar, nic, co])
    return rows

def leading_brand_fix(brand, desc):
    """
    Move leading brand words from desc into brand until a characteristic token appears.
    Handles cases like:
      brand='Benson &' desc='& Hedges King F HP'  -> 'Benson & Hedges' + 'King F HP'
      brand='Best'     desc='Buy* 100 F HP'       -> 'Best Buy*'      + '100 F HP'
      brand='All'      desc='American Value 100'  -> 'All American Value' + '100'
    """
    if not desc:
        return brand, desc, 0
    tokens = desc.split()
    moved = []
    star = 1 if ("*" in brand) else 0

    # move '& Something' or other capitalized words until a characteristic token
    while tokens:
        t = tokens[0]
        upper = re.sub(r'[^A-Za-z0-9&-]', '', t).upper()
        # stop if token is obviously a characteristic or number (e.g., 100)
        if upper in CHAR_STARTERS or NUMTOK_RE.match(t):
            break
        # likely part of brand -> move it
        moved.append(tokens.pop(0))

    if moved:
        brand = (brand + " " + " ".join(moved)).strip()

    # if desc started with "& Hedges" and brand was "Benson &", above will move "&" and "Hedges"
    # remove any '*' that came with moved piece
    if "*" in brand:
        star = 1
        brand = brand.replace("*","").strip()

    desc = " ".join(tokens).strip()
    return brand, desc, star

def parse_characteristics(desc):
    d = (desc or "").strip()

    # Size with no missing: prefer 100, then King; otherwise Reg
    if re.search(r'(?<!\d)100(?!\d)', d):
        size = "100"
    elif re.search(r'(?i)\bKing(s)?\b', d):
        size = "King"
    else:
        size = "Reg"

    # Dummies
    F  = 1 if re.search(r'(?i)(^|[^\w])F($|[^\w])|Full\s*Filter', d) else 0
    NF = 1 if re.search(r'(?i)(^|[^\w])NF($|[^\w])|Non[-\s]*Filter', d) else 0
    HP = 1 if re.search(r'(?i)\bHP\b|Hard\s*Pack', d) else 0
    SP = 1 if re.search(r'(?i)\bSP\b|Soft\s*Pack', d) else 0

    ULTRA = 1 if re.search(r'(?i)Ultra(?:-|\s*)Lt|Ultra Light|Ultima', d) else 0
    LT    = 1 if (not ULTRA) and re.search(r'(?i)\bLt\b|Light', d) else 0
    FF    = 1 if re.search(r'(?i)\bFF\b|Full\s*Flavor', d) else 0

    MENTHOL = 1 if re.search(r'(?i)Menthol', d) else 0
    GENERIC = 1 if re.search(r'(?i)\bGeneric\b', d) else 0
    DLX     = 1 if re.search(r'(?i)\bDLX\b|Deluxe', d) else 0
    SPECIAL = 1 if re.search(r'(?i)\bSpecial\b', d) else 0
    # "Sup Slim" / "Super Slim" / "Superslim" / "Slim"
    SUPSLIM = 1 if re.search(r'(?i)Sup(?:er)?\s*Slim|Superslim|Super[-\s]*Slim', d) else 0
    SLIM    = 1 if (not SUPSLIM) and re.search(r'(?i)\bSlim(s)?\b', d) else 0

    return size, F, NF, HP, SP, ULTRA, LT, FF, MENTHOL, GENERIC, DLX, SPECIAL, SUPSLIM, SLIM

def split_op_val(s):
    s = (s or "").strip()
    op = "<" if s.startswith("<") else ""
    val = s[1:] if op == "<" else s
    return op, val

all_rows = []
with pdfplumber.open(PDF) as pdf:
    for pno in range(PAGE_START, PAGE_END + 1):
        page = pdf.pages[pno-1]
        text = page.extract_text() or ""
        if not text or not HDR_RE.search(text):
            continue

        cuts, header_top, header_bottom = find_header_cuts(page)
        if not cuts:
            continue

        words = page.extract_words(use_text_flow=True, keep_blank_chars=False)
        page_rows = bucketize(words, cuts, header_bottom)
        all_rows.extend(page_rows)

cleaned = []
for brand, desc, Tar, Nic, CO in all_rows:
    if not (brand or desc):
        continue
    # strip spaces
    brand = (brand or "").strip()
    desc  = (desc  or "").strip()

    # move brand fragments from desc -> brand (Benson & | & Hedges, Best | Buy*, etc.)
    brand, desc, star_from_move = leading_brand_fix(brand, desc)

    # manufacturer-tested star: from brand OR anything we moved
    manufacturer_tested = 1 if ("*" in brand or star_from_move) else 0
    brand = brand.replace("*","").strip()

    # drop any header echoes
    if re.match(r'(?i)^brand\s*name$', brand) or re.match(r'(?i)^description$', desc):
        continue

    # parse characteristics (ensure Size has no missing: 100/King/Reg)
    size, F, NF, HP, SP, ULTRA, LT, FF, MENTHOL, GENERIC, DLX, SPECIAL, SUPSLIM, SLIM = parse_characteristics(desc)

    # TNCO split
    Tar_op, Tar_val = split_op_val(Tar)
    Nic_op, Nic_val = split_op_val(Nic)
    CO_op,  CO_val  = split_op_val(CO)

    # Value-brand flag
    value_brand = 1 if brand.upper() in VALUE_BRANDS else 0

    cleaned.append([
        brand, desc, size, F, NF, HP, SP, ULTRA, LT, FF, MENTHOL, GENERIC, DLX, SPECIAL, SUPSLIM, SLIM,
        Tar, Nic, CO, Tar_op, Tar_val, Nic_op, Nic_val, CO_op, CO_val, manufacturer_tested, value_brand
    ])

# write CSV
os.makedirs(os.path.dirname(os.path.abspath(OUT)), exist_ok=True)
with open(OUT, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow([
        "Brand Name","Description","Size","F","NF","HP","SP","Ultra_Lt","Lt","FF","Menthol",
        "Generic","Dlx","Special","SupSlim","Slim",
        "Tar","Nic","CO","Tar_op","Tar_value","Nic_op","Nic_value","CO_op","CO_value",
        "manufacturer_tested","value_brand"
    ])
    w.writerows(cleaned)

print(f"Wrote {OUT} with {len(cleaned)} rows")

Wrote Characteristics/ftc_1998_tnco_clean.csv with 1333 rows


In [111]:
import pandas as pd
alt = pd.read_csv("Characteristics/ftc_1998_tnco_clean.csv",encoding="cp1252")
alt

,Brand Name,Description,Size,F,NF,HP,SP,Ultra_Lt,Lt,FF,...,Nic,CO,Tar_op,Tar_value,Nic_op,Nic_value,CO_op,CO_value,manufacturer_tested,value_brand
0,Carlton,King F HP Ultra-Lt,King,1,0,1,0,1,0,0,...,<0.05,<0.5,<,0.5,<,0.05,<,0.5,0,0
1,Now,100 F HP,100,1,0,1,0,0,0,0,...,<0.05,<0.5,<,0.5,<,0.05,<,0.5,0,0
2,Now,King F HP,King,1,0,1,0,0,0,0,...,<0.05,<0.5,<,0.5,<,0.05,<,0.5,0,0
3,Carlton,King F SP Ultra,King,1,0,0,1,0,0,0,...,0.1,<0.5,<,0.5,NaN,0.10,<,0.5,1,0
4,Carlton,100 F HP Lt Menthol,100,1,0,1,0,0,1,0,...,0.1,1,<,0.5,NaN,0.10,NaN,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,Worth,King F SP FF Menthol,King,1,0,0,1,0,0,1,...,0.8,17,NaN,14.0,NaN,0.80,NaN,17.0,1,0
1329,Worth,King F SP Lt,King,1,0,0,1,0,1,0,...,0.7,13,NaN,10.0,NaN,0.70,NaN,13.0,1,0
1330,Worth,King F SP Lt Menthol,King,1,0,0,1,0,1,0,...,0.6,12,NaN,9.0,NaN,0.60,NaN,12.0,1,0
1331,Worth,King F SP Ultra-Lt,King,1,0,0,1,1,0,0,...,0.3,6,NaN,4.0,NaN,0.30,NaN,6.0,1,0


# $\text{HTML Cleaning}$

In [68]:
html_text = load_html(URL1,LOCAL1)
soup = BeautifulSoup(html_text, "html.parser")
soup

Reading local file Characteristics/Nicotine, Tar, and CO Content of Menthol Cigarette Brands in 1994.htm


<html>
<head>
<script async="" crossorigin="anonymous" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-9977171164675277"></script>
<title>
Nicotine, Tar, and CO Content of Menthol Cigarette Brands in 1994
</title>
</head>
<body>
<center>
<script type="text/javascript"><!--
google_ad_client = "pub-9977171164675277";
/* cigms_top */
google_ad_slot = "9441854520";
google_ad_width = 728;
google_ad_height = 90;
//-->
</script>
<script src="http://pagead2.googlesyndication.com/pagead/show_ads.js" type="text/javascript">
</script>
</center>
<pre>
NICOTINE, TAR, AND CO CONTENT OF DOMESTIC CIGARETTES IN 1994
    (Menthol Brands, sorted by nicotine, tar, and CO)

  NIC   TAR    CO  BRAND NAME             TYPE
  ---   ---   ---  ----------             ----
   .1     1     1  CARLTON                100  F  HP LT
   .1     1     2  CARLTON                KING F  SP ULTRA-LT
   .1     1     2  NOW                    KING F  SP
   .2     2     2  CARLTON             

In [114]:
import pandas as pd
import requests
from requests.exceptions import HTTPError, RequestException, Timeout, SSLError, ConnectionError
from io import StringIO
from typing import List, Union
import os


OUT_CSV  = "Characteristics/ftc_1994_tnco_clean.csv"
# first url should be menthol cigs

URL_MENTHOL = "https://www.econdataus.com/cigms94.html"
URL_REGULAR = "https://econdataus.com/cigra94.html"



# If you prefer to download first, set these to local filenames (e.g., 'cigms94.html')
LOCAL1 = "Characteristics/Nicotine, Tar, and CO Content of Menthol Cigarette Brands in 1994.htm"  # e.g., "cigms94.html"
LOCAL2 = "Characteristics/Nicotine, Tar, and CO Content of Regular Cigarette Brands in 1994.htm"  # e.g., "cigra94.html"


TITLE_RE = re.compile(r'^\s*NICOTINE,\s*TAR,\s*AND\s*CO\s*CONTENT\s*OF\s*DOMESTIC\s*CIGARETTES\s*IN\s*1994\s*$', re.I)
SUB_MENTHOL_RE = re.compile(r'\(.*Menthol\s+Brands.*\)', re.I)
SUB_REGULAR_RE = re.compile(r'\(.*Regular\s+Brands.*\)', re.I)
HEADER_RE = re.compile(r'^\s*NIC\s+TAR\s+CO\s+BRAND\s+NAME\s+TYPE\s*$', re.I)
STOP_RE = re.compile(r'^\s*KEY\b', re.I)  # stop right before KEY section

def load_html(url, local=None):
    if local and os.path.exists(local):
        with open(local, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    }
    r = requests.get(url, headers=headers, timeout=30)
    r.raise_for_status()
    return r.text

def get_all_pre_texts(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    pres = [p.get_text("\n") for p in soup.find_all("pre")]
    if pres:
        return pres
    # fallback: regex if BS fails
    return [m.group(1) for m in re.finditer(r"<pre[^>]*>(.*?)</pre>", html_text, re.S | re.I)]

def slice_block(pre_text, want_menthol: bool):
    """
    From a <pre> text:
      - find TITLE line
      - require the next non-blank line to include (Menthol Brands...) or (Regular Brands...)
      - start from the table HEADER line (NIC TAR CO BRAND NAME TYPE)
      - stop before a line that starts with KEY
    Return the text slice including header; raise if not found.
    """
    lines = pre_text.splitlines()
    n = len(lines)

    # find the title + correct subtitle block
    start_idx = None
    i = 0
    while i < n:
        if TITLE_RE.search(lines[i]):
            # look ahead for the subtitle on subsequent non-empty line(s)
            j = i + 1
            while j < n and not lines[j].strip():
                j += 1
            if j < n:
                ok = (SUB_MENTHOL_RE.search(lines[j]) if want_menthol else SUB_REGULAR_RE.search(lines[j]))
                if ok:
                    # from here, find the header line
                    k = j + 1
                    while k < n and not HEADER_RE.search(lines[k]):
                        k += 1
                    if k < n:
                        # now collect until KEY
                        out = []
                        for t in range(k, n):
                            if STOP_RE.search(lines[t]):
                                break
                            out.append(lines[t])
                        if out:
                            return "\n".join(out)
        i += 1

    raise ValueError("Could not locate the data block with the expected title/subtitle/header.")

def parse_pre_to_df(block_text: str) -> pd.DataFrame:
    # keep header so read_fwf learns column widths
    return pd.read_fwf(StringIO(block_text))

def get_col(df, key):
    key = key.upper()
    for c in df.columns:
        cu = str(c).upper().strip()
        if key == "BRAND NAME":
            if "BRAND" in cu and "NAME" in cu:
                return c
        elif key in {"NIC","TAR","CO","TYPE"}:
            if cu == key or cu.startswith(key):
                return c
    raise KeyError(f"Column {key} not found. Columns: {list(df.columns)}")

def build_clean_df(df_raw: pd.DataFrame, menthol_override=None) -> pd.DataFrame:
    nic_col = get_col(df_raw, "NIC")
    tar_col = get_col(df_raw, "TAR")
    co_col  = get_col(df_raw, "CO")
    brand_col = get_col(df_raw, "BRAND NAME")
    type_col  = get_col(df_raw, "TYPE")

    rows = []
    for _, r in df_raw.iterrows():
        Tar = str(r[tar_col]).strip()
        Nic = str(r[nic_col]).strip()
        CO  = str(r[co_col]).strip()
        Brand = str(r[brand_col]).strip()
        Desc  = str(r[type_col]).strip()

        size, F, NF, HP, SP, ULTRA, LT, FF, MENTHOL, GENERIC, DLX, SPECIAL, SUPSLIM, SLIM = parse_characteristics(Desc)
        if menthol_override is not None:
            MENTHOL = menthol_override

        Tar_op, Tar_val = split_op_val(Tar)
        Nic_op, Nic_val = split_op_val(Nic)
        CO_op,  CO_val  = split_op_val(CO)
        value_brand = 1 if Brand.upper() in VALUE_BRANDS else 0

        rows.append([
            Brand, Desc, size, F, NF, HP, SP, ULTRA, LT, FF, MENTHOL, GENERIC,
            DLX, SPECIAL, SUPSLIM, SLIM,
            Tar, Nic, CO, Tar_op, Tar_val, Nic_op, Nic_val, CO_op, CO_val,
            0,  # manufacturer_tested (no * in HTML listing)
            value_brand
        ])

    cols = [
        "Brand Name","Description","Size","F","NF","HP","SP","Ultra_Lt","Lt","FF","Menthol",
        "Generic","Dlx","Special","SupSlim","Slim",
        "Tar","Nic","CO","Tar_op","Tar_value","Nic_op","Nic_value","CO_op","CO_value",
        "manufacturer_tested","value_brand"
    ]
    return pd.DataFrame(rows, columns=cols)


def run(local1=None, local2=None, out_csv=OUT_CSV):
    # Menthol page
    html1 = load_html(URL_MENTHOL, LOCAL1)
    pre_texts1 = get_all_pre_texts(html1)
    block1 = None
    for t in pre_texts1:
        try:
            block1 = slice_block(t, want_menthol=True)
            break
        except ValueError:
            continue
    if block1 is None:
        raise ValueError("Menthol block not found.")

    # Regular page
    html2 = load_html(URL_REGULAR, LOCAL2)
    pre_texts2 = get_all_pre_texts(html2)
    block2 = None
    for t in pre_texts2:
        try:
            block2 = slice_block(t, want_menthol=False)
            break
        except ValueError:
            continue
    if block2 is None:
        raise ValueError("Regular block not found.")

    df1_raw = parse_pre_to_df(block1)
    df2_raw = parse_pre_to_df(block2)

    df1 = build_clean_df(df1_raw, menthol_override=1)  # ALWAYS Menthol = 1 for first link
    df2 = build_clean_df(df2_raw, menthol_override=0)

    out = pd.concat([df1, df2], ignore_index=True)
    out.to_csv(out_csv, index=False)
    print(f"Wrote {len(out)} rows to {out_csv}")

if __name__ == "__main__":
    run()


Wrote 532 rows to Characteristics/ftc_1994_tnco_clean.csv


In [122]:
both = pd.concat([pd.read_csv(OUT_CSV),pd.read_csv(OUT)])
both.to_csv( "Characteristics/ftc_1994_1998_tnco_clean.csv", index=False)


In [ ]:
both.col

# $\text{UPC Cleaning}$

In [241]:
from pathlib import Path
import csv, re
from collections import defaultdict
import os

# ===== 1) Set your PS2 root once =====
ROOT = Path(r"")  # <- change if needed

#ROOT = os.getcwd()

UPC_PATH =  ROOT / "Dominick's" / "upccig.csv"
FTC_PATH = ROOT / "Characteristics" / "ftc_1994_1998_tnco_clean.csv"

OUT_DIR  = FTC_PATH.parent

print("Reading UPC  :", UPC_PATH)
print("Reading FTC  :", FTC_PATH)
print("Saving to    :", OUT_DIR)

# ===== 2) Helpers =====
ALIASES = {
    r"B & H": "BENSON & HEDGES",
    # r"\bMARL\s*100S\b": "MARLBORO 100",   # example
}

def norm(s: str) -> str:
    s = (s or "").upper()
    for pat, repl in ALIASES.items():
        s = re.sub(pat, repl, s)
    s = re.sub(r"[^\w&]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    return s

    return s

def parse_size(text: str) -> str:
    t = norm(text)
    if re.search(r"(?<!\d)100(?!\d)|\b100S?\b", t): return "100"
    if re.search(r"\bKING(S)?\b", t):               return "King"
    return "Reg"  # no missing

def parse_pack(text: str) -> str:
    t = norm(text)
    if re.search(r"\b(HP|HARD\s*PACK|BOX|BOXES)\b", t): return "HP"
    if re.search(r"\b(SP|SOFT\s*PACK)\b", t):           return "SP"
    return "UNK"  # fall back to brand+size

def parse_char_dummies(text: str):
    t = norm(text)
    menthol = 1 if "MENTHOL" in t else 0
    dlx     = 1 if re.search(r"\b(DLX|DELUXE)\b", t) else 0
    special = 1 if re.search(r"\bSPECIAL\b", t) else 0
    supslim = 1 if re.search(r"\b(SUPER\s*SLIM|SUPSLIM|SUPERSLIM)\b", t) else 0
    slim    = 1 if (not supslim) and re.search(r"\bSLIM(S)?\b", t) else 0
    generic = 1 if re.search(r"\bGENERIC\b", t) else 0
    carton  = 1 if re.search(r"\bCARTON(S)?\b", t) else 0
    single  = 1 if re.search(r"\bSINGLE\b", t) else 0
    pack_kw = 1 if re.search(r"\bPACK\b|\bPK\b", t) else 0
    return menthol, dlx, special, supslim, slim, generic, carton, single, pack_kw

def safe_float(x):
    s = str(x or "").strip()
    if not s: return None
    try: return float(s)
    except:
        try: return float(s.replace(",", ""))
        except: return None

# ---- brand lexicon from FTC (longest match, read with errors='replace') ----
def build_brand_patterns(ftc_csv_path: Path):
    brands = set()
    with ftc_csv_path.open(newline="", encoding="utf-8", errors="replace") as f:
        r = csv.DictReader(f)
        for row in r:
            b = (row.get("Brand Name") or "").strip()
            if b: brands.add(b)
    # a couple common value-brand aliases
    brands |= {"ALL AMERICAN VALUE", "AMERICAN VALUE", "ALL-AMERICAN VALUE"}

    pats = []
    for b in brands:
        tokens = [re.escape(x) for x in norm(b).split()]
        if tokens:
            pats.append((b, re.compile(r"\b" + r"\s*".join(tokens) + r"\b")))
    pats.sort(key=lambda x: len(x[0]), reverse=True)
    return pats

def find_brand(descrip: str, brand_patterns):
    T = norm(descrip)
    for brand, pat in brand_patterns:
        if pat.search(T):
            return brand
    return ""

# ---- FTC aggregates by (brand,size,pack) → (brand,size) → (brand) ----
def load_ftc_aggregates(ftc_csv_path: Path):
    b_sp = defaultdict(lambda: {"tar": [], "nic": [], "co": []})
    b_s  = defaultdict(lambda: {"tar": [], "nic": [], "co": []})
    b    = defaultdict(lambda: {"tar": [], "nic": [], "co": []})

    with ftc_csv_path.open(newline="", encoding="utf-8", errors="replace") as f:
        r = csv.DictReader(f)
        has_val = {"Tar_value","Nic_value","CO_value"}.issubset(r.fieldnames or [])
        for row in r:
            brand = (row.get("Brand Name") or "").strip()
            if not brand: 
                continue
            size  = (row.get("Size") or "Reg").strip()

            hp = str(row.get("HP") or "").strip()
            sp = str(row.get("SP") or "").strip()
            pack = "HP" if hp in ("1","True","TRUE","true") else ("SP" if sp in ("1","True","TRUE","true") else "UNK")

            if has_val:
                tar = safe_float(row.get("Tar_value"))
                nic = safe_float(row.get("Nic_value"))
                co  = safe_float(row.get("CO_value"))
            else:
                def num_from_raw(v):
                    s = (v or "").strip()
                    s = s[1:] if s.startswith("<") else s
                    return safe_float(s)
                tar = num_from_raw(row.get("Tar"))
                nic = num_from_raw(row.get("Nic"))
                co  = num_from_raw(row.get("CO"))

            for name, val in (("tar",tar), ("nic",nic), ("co",co)):
                if val is None: 
                    continue
                b_sp[(brand,size,pack)][name].append(val)
                b_s[(brand,size)][name].append(val)
                b[(brand,)][name].append(val)

    def reduce_means(B):
        out = {}
        for k, vv in B.items():
            def m(x): return sum(x)/len(x) if x else None
            out[k] = (m(vv["tar"]), m(vv["nic"]), m(vv["co"]))
        return out

    return reduce_means(b_sp), reduce_means(b_s), reduce_means(b)

# ===== 3) Clean & merge (reads UPC with errors='replace') =====
def clean_and_merge(UPC_PATH: Path, FTC_PATH: Path, OUT_DIR: Path):
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    out_clean = OUT_DIR / "upccig_clean.csv"
    out_merge = OUT_DIR / "upccig_with_tnco.csv"

    brand_patterns = build_brand_patterns(FTC_PATH)
    by_bsp, by_bs, by_b = load_ftc_aggregates(FTC_PATH)

    with UPC_PATH.open(newline="", encoding="utf-8", errors="replace") as fin, \
         out_clean.open("w", newline="", encoding="utf-8") as fclean, \
         out_merge.open("w", newline="", encoding="utf-8") as fmerge:

        rin = csv.DictReader(fin)
        cols_in = rin.fieldnames or []
        clean_cols = cols_in + ["brand","size","pack","Menthol","Dlx","Special","SupSlim","Slim","Generic","Carton","Single","Pack_kw"]
        rout_clean = csv.DictWriter(fclean, fieldnames=clean_cols)
        rout_clean.writeheader()

        merge_cols = clean_cols + ["tnco_key_level","Tar_mean","Nic_mean","CO_mean"]
        rout_merge = csv.DictWriter(fmerge, fieldnames=merge_cols)
        rout_merge.writeheader()

        n, matched = 0, 0
        for row in rin:
            n += 1
            descr = (row.get("DESCRIP") or row.get("DESCR") or "").strip()

            brand = find_brand(descr, brand_patterns) or ""   # NA if not found
            size  = parse_size(descr)
            pack  = parse_pack(descr)                         # HP / SP / UNK


            ######### HARDCODED bc why not
            if descr == 'NAME-GENERIC CIGS KI':
                size = "King"

            Menthol, Dlx, Special, SupSlim, Slim, Generic, Carton, Single, Pack_kw = parse_char_dummies(descr)

            clean_row = dict(row)
            clean_row.update({
                "brand": brand if brand else "NA",
                "size": size,
                "pack": pack,
                "Menthol": Menthol, "Dlx": Dlx, "Special": Special,
                "SupSlim": SupSlim, "Slim": Slim, "Generic": Generic,
                "Carton": Carton, "Single": Single, "Pack_kw": Pack_kw
            })
            rout_clean.writerow(clean_row)

            # Merge priority: (brand,size,pack) → (brand,size) → (brand)
            tnco_key_level = "none"
            Tar_mean = Nic_mean = CO_mean = ""

            if brand:
                if pack in ("HP","SP") and (brand,size,pack) in by_bsp:
                    Tar_mean, Nic_mean, CO_mean = by_bsp[(brand,size,pack)]; tnco_key_level = "brand+size+pack"
                elif (brand,size) in by_bs:
                    Tar_mean, Nic_mean, CO_mean = by_bs[(brand,size)]; tnco_key_level = "brand+size"
                elif (brand,) in by_b:
                    Tar_mean, Nic_mean, CO_mean = by_b[(brand,)]; tnco_key_level = "brand"

            matched += (tnco_key_level != "none")

            merge_row = dict(clean_row)
            merge_row.update({
                "tnco_key_level": tnco_key_level,
                "Tar_mean": f"{Tar_mean:.3f}" if isinstance(Tar_mean,float) else "",
                "Nic_mean": f"{Nic_mean:.3f}" if isinstance(Nic_mean,float) else "",
                "CO_mean":  f"{CO_mean:.3f}"  if isinstance(CO_mean, float) else "",
            })
            rout_merge.writerow(merge_row)

    print(f"Wrote: {out_clean}")
    print(f"Wrote: {out_merge}")
    print(f"Rows: {n} | matched to FTC at any level: {matched}")

# run
clean_and_merge(UPC_PATH, FTC_PATH, OUT_DIR)


Reading UPC  : Dominick's\upccig.csv
Reading FTC  : Characteristics\ftc_1994_1998_tnco_clean.csv
Saving to    : Characteristics
Wrote: Characteristics\upccig_clean.csv
Wrote: Characteristics\upccig_with_tnco.csv
Rows: 942 | matched to FTC at any level: 168


In [224]:
from pathlib import Path
import csv, re
from collections import defaultdict

# ---- set your PS2 root folder once ----
ROOT = Path(r"F:\Codes\Jupyter\ECON847\PS2")   # <-- adjust if needed
INPATH = ROOT / "Characteristics" / "upccig_with_tnco.csv"
OUTDIR = INPATH.parent

print("Reading:", INPATH)

def is_clear_brand(b):
    b = (b or "").strip().upper()
    return b not in {"", "NA", "N/A", "UNK", "UNKNOWN"}

# find a UPC column name robustly
def pick_upc_col(fieldnames):
    if not fieldnames: return None
    # prefer exact names, then any column containing 'UPC'
    prefs = ["UPC", "UPC12", "UPC_CODE", "UPCNUM", "UPC CODE", "UPCNUMBER"]
    for p in prefs:
        if p in fieldnames: return p
    for fn in fieldnames:
        if re.search(r"\bUPC\b", fn, re.I): return fn
    return None

n_rows = 0
uniq_descr = set()
uniq_upc   = set()
rows_with_brand = 0

# coverage by description / by upc
descr_has_brand = defaultdict(lambda: False)
upc_has_brand   = defaultdict(lambda: False)

with INPATH.open("r", encoding="utf-8", errors="replace", newline="") as f:
    r = csv.DictReader(f)
    fields = r.fieldnames or []
    upc_col = pick_upc_col(fields)
    if upc_col is None:
        print("WARNING: Couldn't find a UPC column; I'll treat all UPCs as blank.")

    for row in r:
        n_rows += 1
        descr = (row.get("DESCRIP") or row.get("DESCR") or "").strip()
        upc   = (row.get(upc_col) if upc_col else "").strip()
        brand = row.get("brand", "")

        uniq_descr.add(descr)
        uniq_upc.add(upc)

        if is_clear_brand(brand):
            rows_with_brand += 1
            descr_has_brand[descr] = True
            upc_has_brand[upc]     = True

# compute ratios
row_ratio     = rows_with_brand / n_rows if n_rows else 0.0
descr_ratio   = (sum(descr_has_brand.values()) / len(uniq_descr)) if uniq_descr else 0.0
upc_ratio     = (sum(upc_has_brand.values())   / len(uniq_upc))   if uniq_upc else 0.0

print(f"Rows total: {n_rows:,}")
print(f"Unique DESCRIP: {len(uniq_descr):,}")
print(f"Unique UPC: {len(uniq_upc):,}")
print(f"Rows with clear brand: {rows_with_brand:,}  ({row_ratio:.2%})")
print(f"DESCRIP coverage (unique descriptions with a clear brand): {descr_ratio:.2%}")
print(f"UPC coverage (unique UPCs with a clear brand): {upc_ratio:.2%}")

Reading: F:\Codes\Jupyter\ECON847\PS2\Characteristics\upccig_with_tnco.csv


FileNotFoundError: [Errno 2] No such file or directory: 'F:\\Codes\\Jupyter\\ECON847\\PS2\\Characteristics\\upccig_with_tnco.csv'